In [ ]:
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import hist
import hist.dask as hda
import dask
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import vector

NanoAODSchema.warn_missing_crossrefs = False

# IGNORE THE WARNING 

In [ ]:
# Datasets:
#/BulkGravToWW_narrow_M-*_13TeV-madgraph/*NanoAODv7*/NANOAODSIM
#/RSGravToWWToWlepWhad_width0p1_M-*_TuneCUETP8M1_13TeV-madgraph-pythia8/*NanoAODv3*/NANOAODSIM


# When having a large number of files it is useful to put the list on a separate file.
# import json

# with open("semileptonic_notebooks/samples.json", 'r') as sample_file:
#     fileset = json.load(sample_file)

# for sample in fileset:
#     print(sample)

# For now we are only testing on a limited number of files so they are just listed here
fileset = {
    'BulkGravToWW': {
        'files': {
            'root://cmsxrootd.fnal.gov//store/mc/RunIISummer16NanoAODv7/BulkGravToWW_narrow_M-1000_13TeV-madgraph/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/100000/D4404DCB-FBF8-C640-87B0-2DA1D5139083.root': "Events",
        },
        'metadata': {
            'is_mc': 'Events',
        },
    },
    'RSGravToWW': {
        'files': {
            'root://cmsxrootd.fnal.gov//store/mc/RunIISummer16NanoAODv3/RSGravToWWToWlepWhad_width0p1_M-1200_TuneCUETP8M1_13TeV-madgraph-pythia8/NANOAODSIM/PUMoriond17_94X_mcRun2_asymptotic_v3-v1/60000/4442437A-52BB-E811-A8DA-90E2BACC5EEC.root': "Events",
          },
        'metadata': {
            'is_mc': 'Events',
        },
    }
}

In [ ]:
# This step takes some time because it is loading the events
test_dataset = 'BulkGravToWW'
events = NanoEventsFactory.from_root(
    fileset[test_dataset]['files'],
    entry_stop = 1000, # this limit the number of events, good to experiment more quickly
    metadata = fileset[test_dataset]['metadata'],
    schemaclass = NanoAODSchema,
    delayed=False,
).events()

In [ ]:
print(Lets get familiar with what is insiede an event of a NanoAOD file:)
events.fields


print(Now we want to learn about the particles in our events)

print(type(events.GenPart))
print(events.GenPart.fields)

### TASK 1: 
Understand the meaning of the GenPart fields that you should have printed. Some are more easy then others. For now you should make sure you understand 'eta', 'mass', 'phi', 'pt', 'genPartIdxMother', 'pdgId'

### TASK 2: 
Can you select the W bosons? Can you print the value of the W boson mass? And plot?


In [ ]:
# Implement here your code

### TASK 3:

Plot also the other kinematic variables

In [ ]:
# Implement here your code

### TASK 4: 
Now identify the quarks coming from the W bosons and plot their kinematic variables


In [ ]:
# Implement here your code 
# This starts to get more complicated! Take your time to think!

#### Bonus task:

Are there many quarks not coming from the Ws? Would your answer be different if you would be using Vector Boson Scattering samples? Remind me about it at some point and we may give it a go :)

## Polarization variable cos theta

To calculate this variable we are putting together some of the things you have been working on before. Two functions have already been implemented for you. Read them below and try to understand what they do. Add all printouts that you need. Also printing the type of the variables is useful. Or running on a small number of events to explore them in depth.

In [ ]:
# This code was written by David https://github.com/wbuitrago/Scripts_HadronicVBS/blob/main/Costheta.py
def get_w_decay_quark_pairs_gen(events):
    """
    Extraction of pairs of quarks coming from a W boson at generator level
    Gave a 'vector.Momentum4D'.
    """
    if events is None or len(events) == 0:
        return []
        
    pdgId = events.GenPart.pdgId
    mothers = events.GenPart.genPartIdxMother
    
    all_quark_pairs_vectors = []

    print(f"[INFO] Processing events to find W -> qq'.")
    num_w_decay_to_quarks = 0
    for i in range(len(events)): 
        event_pdgId = pdgId[i]
        event_mothers = mothers[i]
        
        # Label of the W boson in the actual event
        # abs(pdgId) == 24 for W+ and W-
        w_indices_in_event = ak.local_index(event_pdgId)[abs(event_pdgId) == 24]

        for w_idx in w_indices_in_event:
            # Indexes for the quarks doughters coming from to the W (abs(pdgId) between 1 and 6)
            daughter_indices = ak.local_index(event_pdgId)[
                (event_mothers == w_idx) & (abs(event_pdgId) >= 1) & (abs(event_pdgId) <= 6)
            ]
            
            if len(daughter_indices) == 2:
                num_w_decay_to_quarks += 1
                q1_data = {
                    "pt": events.GenPart.pt[i][daughter_indices[0]],
                    "eta": events.GenPart.eta[i][daughter_indices[0]],
                    "phi": events.GenPart.phi[i][daughter_indices[0]],
                    "mass": events.GenPart.mass[i][daughter_indices[0]],
                }
                q2_data = {
                    "pt": events.GenPart.pt[i][daughter_indices[1]],
                    "eta": events.GenPart.eta[i][daughter_indices[1]],
                    "phi": events.GenPart.phi[i][daughter_indices[1]],
                    "mass": events.GenPart.mass[i][daughter_indices[1]],
                }
                q1_vec = vector.obj(pt=q1_data["pt"], eta=q1_data["eta"], phi=q1_data["phi"], mass=q1_data["mass"])
                q2_vec = vector.obj(pt=q2_data["pt"], eta=q2_data["eta"], phi=q2_data["phi"], mass=q2_data["mass"])
                all_quark_pairs_vectors.append([q1_vec, q2_vec])
                
    print(f"[INFO] Found {len(all_quark_pairs_vectors)} pairs of quarks {num_w_decay_to_quarks} for W->qq'.")
    return all_quark_pairs_vectors


In [ ]:
w_decay_quark_pairs_gen = get_w_decay_quark_pairs_gen(events)

In [ ]:
# This code was written by David https://github.com/wbuitrago/Scripts_HadronicVBS/blob/main/Costheta.py
def calculate_cos_theta_star_gen(quark_pairs_vectors):
    """
    Computation cos(theta*) for the list of pair of quarks.
    theta* is the angle betweeen the quark direction (in the W rest frame) and the dirección of W (in the lab frame).
    """
    cos_theta_stars = []

    if not quark_pairs_vectors:
        print(f"[WARN] Quark pairs not loaded for computation of cos(theta*).")
        return np.array([])
    print(len(quark_pairs_vectors))
    for q_pair in quark_pairs_vectors:
        q1 = q_pair[0] 
        q2 = q_pair[1]
        w_lab = q1 + q2 
        if w_lab.mass < 1e-3 or w_lab.E <= 1e-6:
            print("1")
            continue 

        beta3 = w_lab.to_beta3()
        q1_in_w_rest = q1.boost_beta3(-beta3)
        
        # Axis z' in the helicity frame
        w_direction_lab_3vec = w_lab.to_beta3() 

        if w_direction_lab_3vec.mag < 1e-6:
            continue 
        
        q1_direction_w_rest_3vec = q1_in_w_rest.to_beta3()
        
        if q1_direction_w_rest_3vec.mag < 1e-6: 
            continue

        cos_theta = q1_direction_w_rest_3vec.unit().dot(w_direction_lab_3vec.unit())
        cos_theta_stars.append(cos_theta)
        
    print(f"[INFO] Computed {len(cos_theta_stars)} values of cos(theta*).")
    return np.array(cos_theta_stars)

In [ ]:
cos_theta_star_gen = calculate_cos_theta_star_gen(w_decay_quark_pairs_gen)
print(cos_theta_star_gen)

In [ ]:
gen_cos_theta_star_hist = hist.Hist(hist.axis.StrCategory(name='dataset', label="BG", categories=[], growth=True),
                            hist.axis.Regular(name='gen_AK8_cos_theta_star', label='gen cos_theta_star', bins=80, start=-1, stop=1))
gen_cos_theta_star_hist.fill(dataset=test_dataset, gen_AK8_cos_theta_star=cos_theta_star_gen)
gen_cos_theta_star_hist.plot1d()


### TASK 5:

Can you plot the same variable also for the RSGraviton?

### TASK 6:
Can you plot them on the same figure?

### TASK 7: 

When you are happy with the result, go back to the beginning and add more files with different resonance mass (Let me know when you are here! Not sure it is easy for you to get the list of files!!)

## What we see in our detectors is not the generated particles but it is convolved with their interactions with the detectors or more deacys. For instance Jets instead if Quarks.

Below are two functions implemented for you that match the generated particles with the jets in the event. Read them and try to understand what they do. Add all printouts that you need. Also printing the type of the variables is useful. Or running on a small number of events to explore them in depth.

In [ ]:
# This code was written by David https://github.com/wbuitrago/Scripts_HadronicVBS/blob/main/Costheta.py
def get_w_quark_pairs(events):
    pdgId = events.GenPart.pdgId
    mothers = events.GenPart.genPartIdxMother

    is_w = abs(pdgId) == 24
    is_quark = (abs(pdgId) <= 6) # Quarks u,d,s,c,b,t

    w_indices = ak.local_index(pdgId)[is_w]
    quark_indices = ak.local_index(pdgId)[is_quark]
    quark_parents = mothers[is_quark]

    w_quark_map = {} # (event_index, w_boson_index_in_event) -> [quark1_index_in_event, quark2_index_in_event]

    for evt_idx in range(len(pdgId)):
        # W's in the specific event
        event_w_indices = w_indices[evt_idx]
        # Quarks in the event
        event_quark_indices = quark_indices[evt_idx]
        event_quark_parents = quark_parents[evt_idx]
        
        for w_idx_in_event in event_w_indices:
            # Find quarks coming from W's Parent 
            daughter_quarks_indices_in_event = event_quark_indices[event_quark_parents == w_idx_in_event]
            if len(daughter_quarks_indices_in_event) == 2:
                w_quark_map[(evt_idx, w_idx_in_event)] = daughter_quarks_indices_in_event
    return w_quark_map

In [ ]:
# This code was written by David https://github.com/wbuitrago/Scripts_HadronicVBS/blob/main/Costheta.py
# Matching GenJetAK8 and FatJet with the quark pairs coming from the W decay
def match_w_to_fatjets(events, w_quark_map):
     # Prepare containers for each event (same length as events)
    matched_fatjets_per_event = [[] for _ in range(len(events))]

    genjets = ak.zip({
        "pt": events.GenJetAK8.pt, "eta": events.GenJetAK8.eta,
        "phi": events.GenJetAK8.phi, "mass": events.GenJetAK8.mass
    }, with_name="Momentum4D")

    fatjets = ak.zip({
        "pt": events.FatJet.pt, "eta": events.FatJet.eta,
        "phi": events.FatJet.phi, "mass": events.FatJet.mass,
        "msoftdrop": events.FatJet.msoftdrop,
        "tau1": events.FatJet.tau1, "tau2": events.FatJet.tau2,
        "genJetIdx": events.FatJet.genJetAK8Idx,
        "subJetIdx1": events.FatJet.subJetIdx1,
        "subJetIdx2": events.FatJet.subJetIdx2
    })

    genparts = ak.zip({
        "pt": events.GenPart.pt, "eta": events.GenPart.eta,
        "phi": events.GenPart.phi, "mass": events.GenPart.mass
    }, with_name="Momentum4D")

    for (evt_idx, w_gen_idx), quark_gen_indices in w_quark_map.items():
        q1_gen_idx = quark_gen_indices[0]
        q2_gen_idx = quark_gen_indices[1]

        # Building quarks 4-vector 
        q1 = genparts[evt_idx][q1_gen_idx]
        q2 = genparts[evt_idx][q2_gen_idx]
        
        # Iteration in GenJetAK8 in the event - evt_idx
        for gj_local_idx, gj in enumerate(genjets[evt_idx]):
            if gj.deltaR(q1) < 0.8 and gj.deltaR(q2) < 0.8:
                
                # Mask for the FatJets in the actual event that is matching with gj_local_idx
                matching_fj_mask = (events.FatJet.genJetAK8Idx[evt_idx] == gj_local_idx)
                matched_fjs = events.FatJet[evt_idx][matching_fj_mask]
                
                
                if len(matched_fjs) > 0:
                    matched_fatjets_per_event[evt_idx].append(matched_fjs[0])
                    break

    # Convert list-of-lists of records back into ak.Array
    return ak.Array(matched_fatjets_per_event)

### TASK 8:

Plot on the same plot
1) the mass of the generated Ws
2) the mass of the FatJets
3) the mass of the FatJets coming from a W

Do you understand the plot?

#### Tip: you can run on only one file or with very few events when debugging your code

### TASK 9:

For the FatJets (or the FatJets coming from a W) let's plot also some variables that are commonly used to distinguish Jets coming from a W (or Z boson from the background). Remind me to give you some material to learn about jet tagging when you get here!
1. Compare the mass and the soft drop mass ( `msoftdrop`)
2. Plot also the N-subjettines variables `tau1`, `tau2` and their ratio

### TASK 10:

Identify the subjets of the FatJets coming from the W and plot their kinematic variables

### TASK 11:

Can you calculate cos theta for the jets and subjets instead of the gen particles? This may be tricky!! Take your time!